In [118]:
import numpy as np
import pandas as pd  
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings
from scipy.stats import pearsonr 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.tsa.arima.model import ARIMA
import xgboost as xgb
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional
from math import sqrt
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import tensorflow as tf
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

In [56]:
def json_to_prices(json):
    df = pd.read_json(json)
    prices = df['prices']
    coin_prices = pd.DataFrame(prices.tolist(), columns=['time', 'price'])
    prices = coin_prices['price']
    return prices

In [85]:
def normalize_prices(prices):
    normalized_prices = (prices - np.mean(prices)) / np.std(prices)
    return normalized_prices

def split_data(normalized_prices, split=0.8):
    train_size = int(len(normalized_prices) * split)
    test_size = len(normalized_prices) - train_size
    
    training_data = normalized_prices[0:train_size]
    test_data = normalized_prices[train_size:len(normalized_prices)]
    return training_data, test_data

def create_dataset_helper(data, time_step=1):
    X, y = [], []
    # data = data.reset_index(drop=True)

    for i in range(len(data) - time_step):      
        X.append(data[i:(i + time_step)])
        y.append(data[i + time_step])
    return np.array(X), np.array(y)

def create_dataset(data, time_steps=1):
    X, y = create_dataset_helper(data, time_steps)
    X = X.reshape(X.shape[0], X.shape[1], 1)
    return X, y

# def create_sequences(data, time_steps):
#     X, y = [], []
#     for i in range(time_steps, len(data)):
#         X.append(data[i-time_steps:i])
#         y.append(data[i, 0])  # Assuming the first column is DAI price
#     return np.array(X), np.array(y)

# def create_sequences(data, time_steps):
#     X, y = [], []
#     print(f'data shape: {data.shape}')  # Print the shape of data
#     for i in range(time_steps, len(data)):
#         print(i)  # Print the current index
#         X.append(data[i-time_steps:i])
#         y.append(data[i, 0])  # Assuming the first column is DAI price
#     return np.array(X), np.array(y)

def create_sequences(data, time_steps):
    X, y = [], []
    for i in range(time_steps, len(data)):
        X.append(data[i-time_steps:i].values)  # Use .values to extract as numpy array
        y.append(data.iloc[i, 0])  # Use .iloc to access the data by integer location
    return np.array(X), np.array(y)

In [121]:
def create_model(features, time_step):
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(time_step, features)))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# def create_multivariate_model():
#     model = Sequential()
#     model.add(LSTM(units=50, return_sequences=True, input_shape=(time_step, 3)))
#     model.add(LSTM(units=50))
#     model.add(Dense(1))
#     model.compile(optimizer='adam', loss='mean_squared_error')
#     return model

def predictions(model, X_train, y_train, X_test, y_test, prices):
    train_predictions = model.predict(X_train)
    test_predictions = model.predict(X_test)
    
    train_predict = (train_predictions * np.std(prices)) + np.mean(prices)
    y_train = (y_train * np.std(prices)) + np.mean(prices)
    test_predict = (test_predictions * np.std(prices)) + np.mean(prices)
    y_test = (y_test * np.std(prices)) + np.mean(prices)
    
    train_score = sqrt(mean_squared_error(y_train, train_predict))
    print('Train Score: %.20f RMSE' % (train_score))
    test_score = sqrt(mean_squared_error(y_test, test_predict))
    print('Test Score: %.20f RMSE' % (test_score))
    return train_predict, test_predict
    

In [122]:
# Load the data

dai = json_to_prices('data/dai90days.json')
eth = json_to_prices('data/ethereum90days.json')
usdc = json_to_prices('data/usdc90days.json')

#normalize data

normalized_dai = normalize_prices(dai)
normalized_eth = normalize_prices(eth)
normalized_usdc = normalize_prices(usdc)


In [123]:
# combine the data

combined_data = pd.concat([normalized_dai, normalized_eth, normalized_usdc], axis=1)

In [124]:
print(combined_data)

         price     price     price
0     1.363371 -1.509895  0.313974
1     0.028282 -1.491049 -0.139964
2     0.673200 -1.507219 -0.707387
3     0.189373 -1.519905 -0.026480
4     0.240723 -1.519112 -1.274809
...        ...       ...       ...
2156  1.143258  3.213437 -0.026480
2157  1.341297  3.238812  0.540943
2158  2.757049  3.192427 -1.501778
2159  0.696315  3.212663  0.087005
2160  1.155137       NaN       NaN

[2161 rows x 3 columns]


In [125]:
type(combined_data)

pandas.core.frame.DataFrame

In [126]:
#split the data
time_step = 10

combined_training_data, combined_test_data = split_data(combined_data)

X_train, y_train = create_sequences(combined_training_data, time_step)
X_test, y_test = create_sequences(combined_test_data, time_step)

In [127]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(1718, 10, 3) (1718,) (423, 10, 3) (423,)


In [128]:
dai_multivariate_model = create_model(3, time_step)
dai_multivariate_model.fit(X_train, y_train, epochs=50, batch_size=64)

Epoch 1/50
27/27 [==============================] - 3s 7ms/step - loss: 0.8722
Epoch 2/50
27/27 [==============================] - 0s 7ms/step - loss: 0.8562
Epoch 3/50
27/27 [==============================] - 0s 7ms/step - loss: 0.8415
Epoch 4/50
27/27 [==============================] - 0s 7ms/step - loss: 0.8352
Epoch 5/50
27/27 [==============================] - 0s 7ms/step - loss: 0.8278
Epoch 6/50
27/27 [==============================] - 0s 7ms/step - loss: 0.8239
Epoch 7/50
27/27 [==============================] - 0s 7ms/step - loss: 0.8150
Epoch 8/50
27/27 [==============================] - 0s 7ms/step - loss: 0.8081
Epoch 9/50
27/27 [==============================] - 0s 7ms/step - loss: 0.8017
Epoch 10/50
27/27 [==============================] - 0s 7ms/step - loss: 0.7972
Epoch 11/50
27/27 [==============================] - 0s 7ms/step - loss: 0.7989
Epoch 12/50
27/27 [==============================] - 0s 7ms/step - loss: 0.7951
Epoch 13/50
27/27 [==============================

In [129]:
train_predict, test_predict = predictions(dai_multivariate_model, X_train, y_train, X_test, y_test, dai)

14/14 [==============================] - 0s 3ms/step
Train Score: 0.00092811072173473177 RMSE
Test Score: 0.00142056059492171082 RMSE


In [130]:
# add model to pickle file
import pickle
with open('dai_multivariate_model.pkl', 'wb') as file:
    pickle.dump(dai_multivariate_model, file)


## Ethereum Modeling with Price, Market Cap, and Total Volume

In [131]:
def json_to_market_cap(json):
    df = pd.read_json(json)
    market_caps = df['market_caps']
    coin_market_caps = pd.DataFrame(market_caps.tolist(), columns=['time', 'market_caps'])
    market_caps = coin_market_caps['market_caps']
    return market_caps

def json_to_volume(json):
    df = pd.read_json(json)
    volumes = df['total_volumes']
    coin_volumes = pd.DataFrame(volumes.tolist(), columns=['time', 'volume'])
    volumes = coin_volumes['volume']
    return volumes

In [132]:
prices_1 = json_to_prices('general_model_data/eth_april_june.json')
prices_2 = json_to_prices('general_model_data/eth_june_august.json')
prices_3 = json_to_prices('general_model_data/eth_august_october.json')
prices_4 = json_to_prices('general_model_data/eth_october_december.json')
prices_5 = json_to_prices('general_model_data/eth_december_feb.json')
prices_6 = json_to_prices('general_model_data/eth_feb_march.json')

prices = pd.concat([prices_1, prices_2, prices_3, prices_4, prices_5, prices_6])

#reset the index
prices = prices.reset_index(drop=True)
print(prices)

0       1909.047327
1       1913.866964
2       1888.625330
3       1886.630519
4       1887.228688
           ...     
8036    3583.853827
8037    3612.419591
8038    3601.360086
8039    3617.079283
8040    3605.308078
Name: price, Length: 8041, dtype: float64


In [133]:
market_caps_1 = json_to_market_cap('general_model_data/eth_april_june.json')
market_caps_2 = json_to_market_cap('general_model_data/eth_june_august.json')
market_caps_3 = json_to_market_cap('general_model_data/eth_august_october.json')
market_caps_4 = json_to_market_cap('general_model_data/eth_october_december.json')
market_caps_5 = json_to_market_cap('general_model_data/eth_december_feb.json')
market_caps_6 = json_to_market_cap('general_model_data/eth_feb_march.json')

market_caps = pd.concat([market_caps_1, market_caps_2, market_caps_3, market_caps_4, market_caps_5, market_caps_6])

#reset the index
market_caps = market_caps.reset_index(drop=True)
print(market_caps)

0       2.295167e+11
1       2.303356e+11
2       2.286543e+11
3       2.269938e+11
4       2.271579e+11
            ...     
8036    4.306169e+11
8037    4.348205e+11
8038    4.324637e+11
8039    4.331715e+11
8040    4.328914e+11
Name: market_caps, Length: 8041, dtype: float64


In [134]:
total_volumes_1 = json_to_volume('general_model_data/eth_april_june.json')
total_volumes_2 = json_to_volume('general_model_data/eth_june_august.json')
total_volumes_3 = json_to_volume('general_model_data/eth_august_october.json')
total_volumes_4 = json_to_volume('general_model_data/eth_october_december.json')
total_volumes_5 = json_to_volume('general_model_data/eth_december_feb.json')
total_volumes_6 = json_to_volume('general_model_data/eth_feb_march.json')

total_volumes = pd.concat([total_volumes_1, total_volumes_2, total_volumes_3, total_volumes_4, total_volumes_5, total_volumes_6])

#reset the index
total_volumes = total_volumes.reset_index(drop=True)
print(total_volumes)

0       1.862849e+10
1       1.894294e+10
2       1.894396e+10
3       1.948900e+10
4       1.941160e+10
            ...     
8036    1.967639e+10
8037    1.915767e+10
8038    1.921760e+10
8039    1.886316e+10
8040    1.826426e+10
Name: volume, Length: 8041, dtype: float64


In [135]:
normalized_prices = normalize_prices(prices)
normalized_market_caps = normalize_prices(market_caps)
normalized_volumes = normalize_prices(total_volumes)

In [136]:
combined_data = pd.concat([normalized_prices, normalized_market_caps, normalized_volumes], axis=1)

In [137]:
# drop rows with NaN values
combined_data = combined_data.dropna()
combined_data = combined_data.reset_index(drop=True)

In [138]:
print(combined_data)

         price  market_caps    volume
0    -0.372189    -0.372188  0.710583
1    -0.363606    -0.360036  0.747224
2    -0.408554    -0.384985  0.747342
3    -0.412106    -0.409627  0.810851
4    -0.411041    -0.407192  0.801832
...        ...          ...       ...
8036  2.610128     2.612056  0.832686
8037  2.660995     2.674436  0.772244
8038  2.641301     2.639462  0.779227
8039  2.669292     2.649965  0.737927
8040  2.648331     2.645808  0.668143

[8041 rows x 3 columns]


In [139]:
time_step = 10

combined_training_data, combined_test_data = split_data(combined_data)

X_train, y_train = create_sequences(combined_training_data, time_step)
X_test, y_test = create_sequences(combined_test_data, time_step)

In [140]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(6422, 10, 3) (6422,) (1599, 10, 3) (1599,)


In [141]:
ethereum_model = create_model(3, time_step)
ethereum_model.fit(X_train, y_train, epochs=50, batch_size=64)

Epoch 1/50
101/101 [==============================] - 3s 7ms/step - loss: 0.0131
Epoch 2/50
101/101 [==============================] - 1s 7ms/step - loss: 0.0014
Epoch 3/50
101/101 [==============================] - 1s 7ms/step - loss: 0.0011
Epoch 4/50
101/101 [==============================] - 1s 7ms/step - loss: 0.0011
Epoch 5/50
101/101 [==============================] - 1s 7ms/step - loss: 9.5670e-04
Epoch 6/50
101/101 [==============================] - 1s 7ms/step - loss: 9.1594e-04
Epoch 7/50
101/101 [==============================] - 1s 7ms/step - loss: 9.0798e-04
Epoch 8/50
101/101 [==============================] - 1s 7ms/step - loss: 8.8415e-04
Epoch 9/50
101/101 [==============================] - 1s 7ms/step - loss: 7.7618e-04
Epoch 10/50
101/101 [==============================] - 1s 7ms/step - loss: 7.6300e-04
Epoch 11/50
101/101 [==============================] - 1s 7ms/step - loss: 7.0547e-04
Epoch 12/50
101/101 [==============================] - 1s 7ms/step - loss: 6.75

In [143]:
predictions(ethereum_model, X_train, y_train, X_test, y_test, prices)

50/50 [==============================] - 0s 2ms/step
Train Score: 10.36554028708870411890 RMSE
Test Score: 199.55607401903884579042 RMSE


(array([[1886.5544],
        [1891.383 ],
        [1899.1042],
        ...,
        [2501.322 ],
        [2488.4229],
        [2496.7761]], dtype=float32),
 array([[2479.994 ],
        [2478.5718],
        [2474.1982],
        ...,
        [3355.336 ],
        [3354.273 ],
        [3357.2144]], dtype=float32))

In [144]:
import pickle
with open('ethereum_model.pkl', 'wb') as file:
    pickle.dump(ethereum_model, file)


## USDC Generalized Model with Price, Volume, Market Cap

In [145]:
usdc_prices_1 = json_to_prices('usdc_general_data/usdc_april_june.json')
usdc_prices_2 = json_to_prices('usdc_general_data/usdc_june_aug.json')
usdc_prices_3 = json_to_prices('usdc_general_data/usdc_aug_oct.json')
usdc_prices_4 = json_to_prices('usdc_general_data/usdc_oct_dec.json')
usdc_prices_5 = json_to_prices('usdc_general_data/usdc_dec_feb.json')
usdc_prices_6 = json_to_prices('usdc_general_data/usdc_feb_march.json')

usdc_prices = pd.concat([usdc_prices_1, usdc_prices_2, usdc_prices_3, usdc_prices_4, usdc_prices_5, usdc_prices_6])

prices = usdc_prices.dropna()
all_prices = prices.reset_index(drop=True)
print(all_prices)

0       1.000225
1       1.001516
2       0.999211
3       1.000040
4       1.000291
          ...   
8031    0.999038
8032    1.001351
8033    0.999430
8034    1.000355
8035    0.999473
Name: price, Length: 8036, dtype: float64


In [146]:
usdc_market_caps_1 = json_to_market_cap('usdc_general_data/usdc_april_june.json')
usdc_market_caps_2 = json_to_market_cap('usdc_general_data/usdc_june_aug.json')
usdc_market_caps_3 = json_to_market_cap('usdc_general_data/usdc_aug_oct.json')
usdc_market_caps_4 = json_to_market_cap('usdc_general_data/usdc_oct_dec.json')
usdc_market_caps_5 = json_to_market_cap('usdc_general_data/usdc_dec_feb.json')
usdc_market_caps_6 = json_to_market_cap('usdc_general_data/usdc_feb_march.json')

usdc_market_caps = pd.concat([usdc_market_caps_1, usdc_market_caps_2, usdc_market_caps_3, usdc_market_caps_4, usdc_market_caps_5, usdc_market_caps_6])

market_caps = usdc_market_caps.dropna()
all_market_caps = market_caps.reset_index(drop=True)
print(all_market_caps)

0       3.075954e+10
1       3.078693e+10
2       3.074763e+10
3       3.072970e+10
4       3.073144e+10
            ...     
8031    3.224784e+10
8032    3.236996e+10
8033    3.229050e+10
8034    3.232177e+10
8035    3.229384e+10
Name: market_caps, Length: 8036, dtype: float64


In [147]:
usdc_total_volumes_1 = json_to_volume('usdc_general_data/usdc_april_june.json')
usdc_total_volumes_2 = json_to_volume('usdc_general_data/usdc_june_aug.json')
usdc_total_volumes_3 = json_to_volume('usdc_general_data/usdc_aug_oct.json')
usdc_total_volumes_4 = json_to_volume('usdc_general_data/usdc_oct_dec.json')
usdc_total_volumes_5 = json_to_volume('usdc_general_data/usdc_dec_feb.json')
usdc_total_volumes_6 = json_to_volume('usdc_general_data/usdc_feb_march.json')

usdc_total_volumes = pd.concat([usdc_total_volumes_1, usdc_total_volumes_2, usdc_total_volumes_3, usdc_total_volumes_4, usdc_total_volumes_5, usdc_total_volumes_6])

total_volumes = usdc_total_volumes.dropna()
all_total_volumes = total_volumes.reset_index(drop=True)
print(all_total_volumes)

0       6.614025e+09
1       6.630742e+09
2       6.581552e+09
3       6.729557e+09
4       6.743844e+09
            ...     
8031    8.559277e+09
8032    8.729863e+09
8033    7.669309e+09
8034    6.677742e+09
8035    9.220244e+09
Name: volume, Length: 8036, dtype: float64


In [148]:
normalized_prices = normalize_prices(all_prices)
normalized_market_caps = normalize_prices(all_market_caps)
normalized_volumes = normalize_prices(all_total_volumes)

combined_data = pd.concat([normalized_prices, normalized_market_caps, normalized_volumes], axis=1)

# drop rows with NaN values
combined_data = combined_data.dropna()
combined_data = combined_data.reset_index(drop=True)

print(combined_data)

         price  market_caps    volume
0     0.244814     1.957893  0.093174
1     1.668984     1.971594  0.097332
2    -0.873066     1.951935  0.085097
3     0.041495     1.942966  0.121910
4     0.318060     1.943836  0.125463
...        ...          ...       ...
8031 -1.063126     2.702468  0.577001
8032  1.487375     2.763566  0.619429
8033 -0.630926     2.723814  0.355646
8034  0.389005     2.739454  0.109022
8035 -0.584036     2.725481  0.741398

[8036 rows x 3 columns]


In [149]:
time_step = 10

combined_training_data, combined_test_data = split_data(combined_data)

X_train, y_train = create_sequences(combined_training_data, time_step)
X_test, y_test = create_sequences(combined_test_data, time_step)

In [150]:
usdc_model = create_model(3, time_step)
usdc_model.fit(X_train, y_train, epochs=50, batch_size=64)

Epoch 1/50
101/101 [==============================] - 4s 8ms/step - loss: 0.7513
Epoch 2/50
101/101 [==============================] - 1s 8ms/step - loss: 0.7501
Epoch 3/50
101/101 [==============================] - 1s 7ms/step - loss: 0.7483
Epoch 4/50
101/101 [==============================] - 1s 7ms/step - loss: 0.7480
Epoch 5/50
101/101 [==============================] - 1s 7ms/step - loss: 0.7472
Epoch 6/50
101/101 [==============================] - 1s 7ms/step - loss: 0.7477
Epoch 7/50
101/101 [==============================] - 1s 7ms/step - loss: 0.7480
Epoch 8/50
101/101 [==============================] - 1s 7ms/step - loss: 0.7475
Epoch 9/50
101/101 [==============================] - 1s 8ms/step - loss: 0.7465
Epoch 10/50
101/101 [==============================] - 1s 7ms/step - loss: 0.7463
Epoch 11/50
101/101 [==============================] - 1s 8ms/step - loss: 0.7468
Epoch 12/50
101/101 [==============================] - 1s 7ms/step - loss: 0.7463
Epoch 13/50
101/101 [====

In [151]:
predictions(usdc_model, X_train, y_train, X_test, y_test, all_prices)

50/50 [==============================] - 0s 2ms/step
Train Score: 0.00074499933187780983 RMSE
Test Score: 0.00136117079298918742 RMSE


(array([[1.0000317 ],
        [1.0001844 ],
        [0.9999051 ],
        ...,
        [0.99999505],
        [1.0001348 ],
        [1.000056  ]], dtype=float32),
 array([[1.0000049],
        [1.0000014],
        [1.000011 ],
        ...,
        [1.0000411],
        [1.0004889],
        [1.0002958]], dtype=float32))

In [152]:
import pickle
with open('usdc_model.pkl', 'wb') as file:
    pickle.dump(usdc_model, file)

## DAI Generalized Model with Price, Volume, Market Cap

In [154]:
dai_price_1 = json_to_prices('dai_general_data/dai_april_june.json')
dai_price_2 = json_to_prices('dai_general_data/dai_june_aug.json')  
dai_price_3 = json_to_prices('dai_general_data/dai_aug_oct.json')
dai_price_4 = json_to_prices('dai_general_data/dai_oct_dec.json')
dai_price_5 = json_to_prices('dai_general_data/dai_dec_feb.json')
dai_price_6 = json_to_prices('dai_general_data/dai_feb_march.json')

dai_prices = pd.concat([dai_price_1, dai_price_2, dai_price_3, dai_price_4, dai_price_5, dai_price_6])

prices = dai_prices.dropna()
all_prices = prices.reset_index(drop=True)

dai_market_caps_1 = json_to_market_cap('dai_general_data/dai_april_june.json')
dai_market_caps_2 = json_to_market_cap('dai_general_data/dai_june_aug.json')
dai_market_caps_3 = json_to_market_cap('dai_general_data/dai_aug_oct.json')
dai_market_caps_4 = json_to_market_cap('dai_general_data/dai_oct_dec.json')
dai_market_caps_5 = json_to_market_cap('dai_general_data/dai_dec_feb.json')
dai_market_caps_6 = json_to_market_cap('dai_general_data/dai_feb_march.json')

dai_market_caps = pd.concat([dai_market_caps_1, dai_market_caps_2, dai_market_caps_3, dai_market_caps_4, dai_market_caps_5, dai_market_caps_6])

market_caps = dai_market_caps.dropna()
all_market_caps = market_caps.reset_index(drop=True)

dai_total_volumes_1 = json_to_volume('dai_general_data/dai_april_june.json')
dai_total_volumes_2 = json_to_volume('dai_general_data/dai_june_aug.json')
dai_total_volumes_3 = json_to_volume('dai_general_data/dai_aug_oct.json')
dai_total_volumes_4 = json_to_volume('dai_general_data/dai_oct_dec.json')
dai_total_volumes_5 = json_to_volume('dai_general_data/dai_dec_feb.json')
dai_total_volumes_6 = json_to_volume('dai_general_data/dai_feb_march.json')

dai_total_volumes = pd.concat([dai_total_volumes_1, dai_total_volumes_2, dai_total_volumes_3, dai_total_volumes_4, dai_total_volumes_5, dai_total_volumes_6])

total_volumes = dai_total_volumes.dropna()
all_total_volumes = total_volumes.reset_index(drop=True)

normalized_prices = normalize_prices(all_prices)
normalized_market_caps = normalize_prices(all_market_caps)
normalized_volumes = normalize_prices(all_total_volumes)

combined_data = pd.concat([normalized_prices, normalized_market_caps, normalized_volumes], axis=1)

# drop rows with NaN values
combined_data = combined_data.dropna()
combined_data = combined_data.reset_index(drop=True)

time_step = 10

combined_training_data, combined_test_data = split_data(combined_data)

X_train, y_train = create_sequences(combined_training_data, time_step)
X_test, y_test = create_sequences(combined_test_data, time_step)

dai_price_model = create_model(3, time_step)
dai_price_model.fit(X_train, y_train, epochs=50, batch_size=64)

predictions(dai_price_model, X_train, y_train, X_test, y_test, all_prices)

Epoch 1/50
101/101 [==============================] - 4s 7ms/step - loss: 0.6845
Epoch 2/50
101/101 [==============================] - 1s 8ms/step - loss: 0.6616
Epoch 3/50
101/101 [==============================] - 1s 7ms/step - loss: 0.6532
Epoch 4/50
101/101 [==============================] - 1s 7ms/step - loss: 0.6509
Epoch 5/50
101/101 [==============================] - 1s 7ms/step - loss: 0.6419
Epoch 6/50
101/101 [==============================] - 1s 7ms/step - loss: 0.6427
Epoch 7/50
101/101 [==============================] - 1s 7ms/step - loss: 0.6417
Epoch 8/50
101/101 [==============================] - 1s 7ms/step - loss: 0.6381
Epoch 9/50
101/101 [==============================] - 1s 7ms/step - loss: 0.6380
Epoch 10/50
101/101 [==============================] - 1s 7ms/step - loss: 0.6364
Epoch 11/50
101/101 [==============================] - 1s 7ms/step - loss: 0.6347
Epoch 12/50
101/101 [==============================] - 1s 7ms/step - loss: 0.6349
Epoch 13/50
101/101 [====

(array([[0.99977475],
        [0.9998273 ],
        [0.99978286],
        ...,
        [0.9985471 ],
        [0.9985286 ],
        [0.99898535]], dtype=float32),
 array([[0.9992569 ],
        [0.99940795],
        [0.9994439 ],
        ...,
        [0.99955523],
        [1.0003151 ],
        [1.0004244 ]], dtype=float32))

In [155]:
import pickle
with open('dai_price_model.pkl', 'wb') as file:
    pickle.dump(dai_price_model, file)

## USDT Generalized Model with Price, Volume, Market Cap

In [160]:
usdt_prices_1 = json_to_prices('usdt_general_data/tether_april_june.json')
usdt_prices_2 = json_to_prices('usdt_general_data/tether_june_aug.json')
usdt_prices_3 = json_to_prices('usdt_general_data/tether_aug_oct.json')
usdt_prices_4 = json_to_prices('usdt_general_data/tether_oct_dec.json')
usdt_prices_5 = json_to_prices('usdt_general_data/tether_dec_feb.json')
usdt_prices_6 = json_to_prices('usdt_general_data/tether_feb_march.json')

usdt_prices = pd.concat([usdt_prices_1, usdt_prices_2, usdt_prices_3, usdt_prices_4, usdt_prices_5, usdt_prices_6])

prices = usdt_prices.dropna()
all_prices = prices.reset_index(drop=True)

usdt_market_caps_1 = json_to_market_cap('usdt_general_data/tether_april_june.json')
usdt_market_caps_2 = json_to_market_cap('usdt_general_data/tether_june_aug.json')
usdt_market_caps_3 = json_to_market_cap('usdt_general_data/tether_aug_oct.json')
usdt_market_caps_4 = json_to_market_cap('usdt_general_data/tether_oct_dec.json')
usdt_market_caps_5 = json_to_market_cap('usdt_general_data/tether_dec_feb.json')
usdt_market_caps_6 = json_to_market_cap('usdt_general_data/tether_feb_march.json')

usdt_market_caps = pd.concat([usdt_market_caps_1, usdt_market_caps_2, usdt_market_caps_3, usdt_market_caps_4, usdt_market_caps_5, usdt_market_caps_6])

market_caps = usdt_market_caps.dropna()
all_market_caps = market_caps.reset_index(drop=True)

usdt_total_volumes_1 = json_to_volume('usdt_general_data/tether_april_june.json')
usdt_total_volumes_2 = json_to_volume('usdt_general_data/tether_june_aug.json')
usdt_total_volumes_3 = json_to_volume('usdt_general_data/tether_aug_oct.json')
usdt_total_volumes_4 = json_to_volume('usdt_general_data/tether_oct_dec.json')
usdt_total_volumes_5 = json_to_volume('usdt_general_data/tether_dec_feb.json')
usdt_total_volumes_6 = json_to_volume('usdt_general_data/tether_feb_march.json')

usdt_total_volumes = pd.concat([usdt_total_volumes_1, usdt_total_volumes_2, usdt_total_volumes_3, usdt_total_volumes_4, usdt_total_volumes_5, usdt_total_volumes_6])

total_volumes = usdt_total_volumes.dropna()
all_total_volumes = total_volumes.reset_index(drop=True)


normalized_prices = normalize_prices(all_prices)
normalized_market_caps = normalize_prices(all_market_caps)
normalized_volumes = normalize_prices(all_total_volumes)

combined_data = pd.concat([normalized_prices, normalized_market_caps, normalized_volumes], axis=1)

# drop rows with NaN values
combined_data = combined_data.dropna()
combined_data = combined_data.reset_index(drop=True)

time_step = 10

combined_training_data, combined_test_data = split_data(combined_data)

X_train, y_train = create_sequences(combined_training_data, time_step)
X_test, y_test = create_sequences(combined_test_data, time_step)

usdt_model = create_model(3, time_step)
usdt_model.fit(X_train, y_train, epochs=50, batch_size=64)

predictions(usdt_model, X_train, y_train, X_test, y_test, all_prices)

Epoch 1/50
101/101 [==============================] - 4s 7ms/step - loss: 0.5863
Epoch 2/50
101/101 [==============================] - 1s 7ms/step - loss: 0.5687
Epoch 3/50
101/101 [==============================] - 1s 8ms/step - loss: 0.5657
Epoch 4/50
101/101 [==============================] - 1s 9ms/step - loss: 0.5641
Epoch 5/50
101/101 [==============================] - 1s 8ms/step - loss: 0.5653
Epoch 6/50
101/101 [==============================] - 1s 7ms/step - loss: 0.5621
Epoch 7/50
101/101 [==============================] - 1s 8ms/step - loss: 0.5620
Epoch 8/50
101/101 [==============================] - 1s 7ms/step - loss: 0.5605
Epoch 9/50
101/101 [==============================] - 1s 7ms/step - loss: 0.5616
Epoch 10/50
101/101 [==============================] - 1s 7ms/step - loss: 0.5616
Epoch 11/50
101/101 [==============================] - 1s 7ms/step - loss: 0.5601
Epoch 12/50
101/101 [==============================] - 1s 7ms/step - loss: 0.5589
Epoch 13/50
101/101 [====

(array([[1.0004435 ],
        [1.0004016 ],
        [1.000385  ],
        ...,
        [0.999188  ],
        [0.99916357],
        [0.9991735 ]], dtype=float32),
 array([[0.999182  ],
        [0.9991957 ],
        [0.99916846],
        ...,
        [0.9992509 ],
        [0.9991336 ],
        [0.9992691 ]], dtype=float32))

In [161]:
import pickle
with open('usdt_model.pkl', 'wb') as file:
    pickle.dump(usdt_model, file)

In [162]:
# load model back from pickle file
with open('usdt_model.pkl', 'rb') as file:
    usdt_model = pickle.load(file)
    
print(usdt_model.summary())

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_16 (LSTM)              (None, 10, 50)            10800     
                                                                 
 lstm_17 (LSTM)              (None, 50)                20200     
                                                                 
 dense_8 (Dense)             (None, 1)                 51        
                                                                 
Total params: 31051 (121.29 KB)
Trainable params: 31051 (121.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
